In [1]:
import pipeline

%matplotlib inline
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import pickle

import PIL.Image
from IPython.display import display
import time
from datetime import timedelta
from sklearn.model_selection import train_test_split
from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input
from keras.models import model_from_json
from keras import regularizers
from tensorflow.keras import optimizers
from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.


In [2]:
directory = '/home/devon/Desktop/BirdPi/images/'
num_species = 17 
image_resize = 299

# Creates a pickle and returns the names of the species
image_labels = pipeline.make_image_label_pickle(directory, num_species, nwidth=image_resize, 
                                                nheight=image_resize)

MemoryError: 

In [ ]:
images = pickle.load(open("images.p", "rb"))
rand_bird = np.random.randint(low=0, high=len(images))
plt.imshow(images[rand_bird,:,:,:]) # Fun visualization showing how each bird image looks to the model at the start
plt.show()
print("There are {} species in this dataset.".format(num_species))
print("There are {} individual images in the dataset.".format(len(images)))

In [ ]:
num_validation = 0.10
X_train, X_validation, y_train, y_validation = train_test_split(images, image_labels, 
                                                                test_size=num_validation, random_state=6)

onehot_encoder = OneHotEncoder(sparse=False, categories='auto') # Creates a code/cypher for what species the bird is as a list of 1s and 0s
y_train_temp = y_train.reshape(len(y_train), 1)                 
y_train = onehot_encoder.fit_transform(y_train_temp)
print(y_train[0])

y_validation_temp = y_validation.reshape(len(y_validation), 1)
y_validation = onehot_encoder.fit_transform(y_validation_temp)
print(y_validation[0])

In [ ]:
img_shape = (image_resize, image_resize, 3)
# Load the unedited model
base_model = tf.keras.applications.Xception(
    input_shape=img_shape,
    include_top=False)

In [ ]:
# Freeze the pre-trained model weights
base_model.trainable = False
# Trainable classification head
maxpool_layer = tf.keras.layers.GlobalMaxPooling2D(input_shape=base_model.output_shape[1:])
prediction_layer = tf.keras.layers.Dense(17, activation='softmax')
dropout = tf.keras.layers.Dropout(0.2)
dense512 = tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01))
dense256 = tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01))
# Layer classification head with feature detector
model = tf.keras.Sequential([base_model,
                             maxpool_layer,
                             dense512,
                             dropout,
                             dense256,
                             dropout,
                             prediction_layer])
learning_rate = 0.0001
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
print("Going birdwatching...")
checkpointer = ModelCheckpoint(filepath='weights.hdf5', verbose=1, save_best_only=True)
model.fit(X_train, y_train, epochs=100, validation_data=(X_validation, y_validation),
          batch_size=20, callbacks=[checkpointer])

In [ ]:
# Save our finished model to a .json
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")
print("Saved model to EC2")

In [ ]:
model.save("model.h5")

In [ ]:
img_path = '/home/ubuntu/images/Rock_Pidgeon/c270436e-b9f8-11e9-b195-7b64bbd6a9bc.jpg'
img = pp.image.load_img(path=img_path, target_size=(155, 155))
x = pp.image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = app.xception.preprocess_input(x)

preds = model.predict(x)
print('Predicted:', app.xception.decode_predictions(preds))